<a href="https://colab.research.google.com/github/anruki/Procesamiento_YOLO/blob/main/p3_Miguel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install opencv-python ultralytics

In [6]:
from ultralytics import YOLO
import cv2

In [7]:
from google.colab.patches import cv2_imshow


In [8]:


# Cargar modelo YOLOv8
model = YOLO('yolov8n.pt')

# Ruta del video de entrada
video_path = 'caballo.mp4'

# Cargar el video
cap = cv2.VideoCapture(video_path)

# Obtener video procesado
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'caballo_procesado.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

ret = True

# Procesar frames
while ret:
    ret, frame = cap.read()
    if ret:
        # Detectar y seguir objetos
        results = model.track(frame, persist=True)

        # Dibujar resultados en el frame
        frame_ = results[0].plot()

        # Escribir el frame procesado en el archivo de salida
        out.write(frame_)

# Liberar recursos
cap.release()
out.release()

print(f"Video procesado guardado en {output_path}")


100%|██████████| 6.25M/6.25M [00:00<00:00, 66.0MB/s]



0: 384x640 1 truck, 1 horse, 1 cow, 359.0ms
Speed: 6.5ms preprocess, 359.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 cow, 351.6ms
Speed: 5.0ms preprocess, 351.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 725.3ms
Speed: 9.8ms preprocess, 725.3ms inference, 10.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 685.1ms
Speed: 6.0ms preprocess, 685.1ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 542.3ms
Speed: 6.6ms preprocess, 542.3ms inference, 17.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 717.8ms
Speed: 5.8ms preprocess, 717.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 567.2ms
Speed: 5.1ms preprocess, 567.2ms inference, 1.6ms postprocess per imag